In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3  # Import InceptionV3 instead of DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import wandb
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # This line prevents TensorFlow from using GPU

# Initialize wandb and config
wandb.init(
    project="computer_vision",
    config={
        "learning_rate": 0.0005,
        "architecture": "InceptionV3",  # Update architecture name
        "dataset": "alzheimer",
        "epochs": 100,
        "batch_size": 16,
        "input_shape": (299, 299, 3),  # Update to match InceptionV3 input size
        "num_classes": 4
    }
)

config = wandb.config

# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=config.input_shape[:2],
    batch_size=config.batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'test',
    target_size=config.input_shape[:2],
    batch_size=config.batch_size,
    class_mode='categorical'
)

# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weight_dict = dict(enumerate(class_weights))

# Create the InceptionV3 model
def create_inceptionv3_model(input_shape, num_classes, dropout_rate=0.2):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    # Freeze all layers except for the last few
    for layer in base_model.layers[:-10]:
        layer.trainable = False

    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Initialize InceptionV3 model
model = create_inceptionv3_model(config.input_shape, config.num_classes)

# Create an Adam optimizer
optimizer = Adam(
    learning_rate=config.learning_rate,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-8
)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Print and log model summary
model.summary()
wandb.log({'model_summary': model.summary()})

# Create a checkpoint callback
checkpoint_callback = ModelCheckpoint(
    'model/best_model_InceptionV3_{epoch:02d}_{val_accuracy:.4f}.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    epochs=config.epochs,
    validation_data=test_generator,
    class_weight=class_weight_dict,
    callbacks=[checkpoint_callback]
)

# Plot training & validation loss/accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid(True)
plt.title('Training vs. Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.grid(True)
plt.title('Training vs. Validation Loss')

plt.tight_layout()
plt.show()

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
wandb.log({'test_loss': test_loss, 'test_accuracy': test_acc})
print(f'Test accuracy: {test_acc:.4f}')

# Predict class probabilities for the test set using the trained model
y_pred_probs = model.predict(test_generator)
# Determine the predicted class by taking the index of the highest probability
y_pred = np.argmax(y_pred_probs, axis=-1)

# Retrieve the actual class labels
y_true = test_generator.classes

# Generate the confusion matrix based on true and predicted labels
cm = confusion_matrix(y_true, y_pred)
# Retrieve the list of class names from the test generator
labels = list(test_generator.class_indices.keys())

# Create a ConfusionMatrixDisplay object to visualize the matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
# Plot the confusion matrix using a blue colormap
disp.plot(cmap=plt.cm.Blues)
plt.show()

# Save the model using the final accuracy
model_filename = f'alzheimer_model_inceptionv3_{test_acc:.4f}.keras'
model.save(os.path.join('model', model_filename))
print(f"Model saved successfully as '{model_filename}'.")

2024-05-08 15:13:22.375446: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Currently logged in as: oskar-amarkussen (ossyandlars). Use `wandb login --relogin` to force relogin


Found 5121 images belonging to 4 classes.
Found 1291 images belonging to 4 classes.


2024-05-08 15:13:27.702077: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

...

 Non-trainable params: 21,802,592 (83.17 MB)

Epoch 1/100


/home/oskarem/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


321/321 ━━━━━━━━━━━━━━━━━━━━ 0s 525ms/step - accuracy: 0.3717 - loss: 1.5749
Epoch 1: val_accuracy improved from -inf to 0.27808, saving model to model/best_model_InceptionV3_01_0.2781.keras
321/321 ━━━━━━━━━━━━━━━━━━━━ 220s 665ms/step - accuracy: 0.3718 - loss: 1.5742 - val_accuracy: 0.2781 - val_loss: 1.3453
Epoch 2/100
321/321 ━━━━━━━━━━━━━━━━━━━━ 0s 543ms/step - accuracy: 0.4831 - loss: 1.0402
Epoch 2: val_accuracy did not improve from 0.27808
321/321 ━━━━━━━━━━━━━━━━━━━━ 216s 672ms/step - accuracy: 0.4832 - loss: 1.0401 - val_accuracy: 0.2239 - val_loss: 1.4550
Epoch 3/100
321/321 ━━━━━━━━━━━━━━━━━━━━ 0s 533ms/step - accuracy: 0.5134 - loss: 0.9268
Epoch 3: val_accuracy improved from 0.27808 to 0.37800, saving model to model/best_model_InceptionV3_03_0.3780.keras
321/321 ━━━━━━━━━━━━━━━━━━━━ 213s 663ms/step - accuracy: 0.5134 - loss: 0.9268 - val_accuracy: 0.3780 - val_loss: 1.2882
Epoch 4/100
321/321 ━━━━━━━━━━━━━━━━━━━━ 0s 536ms/step - accuracy: 0.5136 - loss: 0.8383
Epoch 4: va

KeyboardInterrupt: 